In [2]:
from gtfs_functions import Feed
import logging
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import h3
import pydeck as pdk
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from src.civicsim.common.utils import get_census_data
import src.civicsim.common.constants as constants
from src.civicsim.router.osmx_router import OSMX_Router
from shapely.geometry import Point, LineString

In [4]:
# Load BOC area
boc_gpd = gpd.read_file("./data/BOC.geojson")

# Load Target G
G = ox.load_graphml("./data/target_area.graphml")

# Load user requests
all_user_requests = gpd.read_file("./data/generated_user_requests_from_population.geojson")
all_user_requests['COUNTYFP'] = all_user_requests['GEOID'].str.slice(2,5)
all_user_requests['COUNTY'] = all_user_requests['COUNTYFP'].map(constants.FPS_COUNTY_MAP)

,name,geometry
0,BOC,"LINESTRING (-89.40586 35.43941, -89.45296 35.4..."
1,visitor center,POINT (-89.41953 35.39128)
2,performance center,POINT (-89.42044 35.39726)
3,water tower,POINT (-89.43620 35.41712)
4,proposed supplier park,POINT (-89.43692 35.42303)


### Read all routes created (end to end)
* Plot using h3 geos
* Generated by running `nohup python notebooks/getting_all_route_pairs.py &`

In [5]:
from shapely import wkt
import ast

all_routes_df = pd.read_csv("./output/length_routes.txt", delimiter=';', header=None, names=["id", "o", "d", "r"])
all_routes_df['o'] = all_routes_df['o'].apply(wkt.loads)
all_routes_df['d'] = all_routes_df['d'].apply(wkt.loads)
all_routes_df['r'] = all_routes_df['r'].apply(ast.literal_eval)

passed_nodes = []
for k, v in all_routes_df.iterrows():
    r = v['r']
    passed_nodes.extend(r)
len(passed_nodes)

951970

In [19]:
def value_to_color(value, min_value, max_value):
    # Normalize the value to a range between 0 and 1
    normalized = (value - min_value) / (max_value - min_value)

    # Interpolate between blue and red
    red = normalized * 255       # As the value goes up, red goes from 0 to 255
    green = 0                    # Green is always 0 in the gradient from blue to red
    blue = (1 - normalized) * 255 # As the value goes up, blue goes from 255 to 0

    # Ensure RGB values are integers since color channels range from 0 to 255
    return (int(red), int(green), int(blue))

In [23]:
resolution = 6.5
passed_nodes_unique = list(set(passed_nodes))
node_to_h3_map = {}

for node in passed_nodes_unique:
    # print(node)
    n = G.nodes[node]
    lng = n['x']
    lat = n['y']
    _h = h3.geo_to_h3(lat, lng, resolution)
    node_to_h3_map[node] = _h

pndf = pd.DataFrame({"nodes": passed_nodes})
pndf = pndf.reset_index()
pndf = pndf.groupby("nodes").count().reset_index()
pndf['nodes'] = pndf['nodes'].map(node_to_h3_map)
pndf['color'] = pndf['index'].apply(lambda x: value_to_color(x, 1, 10339))
pndf = pndf[(pndf['index'] > 20) & (pndf['index'] < 500)]

In [25]:
lat, lng = 35.39128, -89.41953

# Define a layer to display on a map
layer = pdk.Layer(
    "H3HexagonLayer",
    pndf,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    get_hexagon="nodes",
    get_fill_color="[255 - index, 255, index]",
    # get_fill_color="color",
    get_line_color=[100,100,100],
    line_width_min_pixels=2,
)

# Define the data for the marker
data = [{"position": [lng, lat], "name": "BOC"}]

# Create the ScatterplotLayer
boc_layer = pdk.Layer(
    "ScatterplotLayer",
    data=data,
    get_position="position",
    get_radius=2000,  # Size of the marker in meters
    get_color=[255, 0, 0],  # Marker color (RGB)
)


# Set the viewport location
view_state = pdk.ViewState(latitude=lat, longitude=lng, zoom=11, bearing=0, pitch=0)


# Render
r = pdk.Deck(layers=[layer, boc_layer], initial_view_state=view_state, tooltip={"text": "Density: {index}"})
r.to_html("h3_cluster_layer.html")
